In [1]:
%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
%jsroot on

/home/yoren/.local/lib/python3.10/site-packages/nbformat/__init__.py:96: MissingIDFieldWarning: Cell is missing an id field, this will become a hard error in future nbformat versions. You may want to use `normalize()` on your notebooks before validations (available since nbformat 5.1.4). Previous versions of nbformat are fixing this issue transparently, and will stop doing so in the future.
  validate(nb)


Welcome to JupyROOT 6.30/06


In [2]:
Save_to_html = False

In [3]:
N_centr = 6
N_sectors = 8
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins0 = [0,100,0,20,20,40,40,60,60,80,80,100]
central_bins = [0,80,0,16,16,32,32,48,48,64,64,80]
pt_boarders = [0.5,0.9,0.2,0.5,0.5,1.0,1.0,5.0,0.5,5]
file_path="../VTXalignment/input/"
file_names=["my-m_ee_Run14AuAu_140th_new_20009_1064runs.root", "my-m_ee_Run14AuAu_140th_new_20009_1064runs.root"] # m_ee_Run14AuAu_140th_new_20009_1064runs
file_names=["my-m_ee_Run14AuAu_143rd_new_20013_1065runs.root"] # m_ee_Run14AuAu_142nd_new_20012_700runs m_ee_Run14AuAu_143rd_new_20013_100runs
sim_file_path="../sim/output/summer25/brem/"
sim_file_names=['jpsi_v09_reg.root']#jpsi_v06_300mev_nomass_newcode1 jpsi_v06_300mev_nomass_newcode
sim_nc_file_names=['jpsi_v09_nc.root']
sim_phi0_file_names = ['jpsi_v04_300mev_phi0.root']
sim_file_names = [sim_file_names[0],sim_nc_file_names[0]]
legend_names = ["data","preselcted","sim","sim_pres"] 
hist_select_3D_names = ["hist_bremstrahlung_phi","hist_bremstrahlung_the","hist_bremstrahlung_e"]
hist_sim_3D_names = ["hist_bremstrahlung_phi","hist_bremstrahlung_the","hist_bremstrahlung_e","hist_sing_pt"]

In [4]:
N_hists = len(hist_select_3D_names)

In [5]:
hists_read, hists_sim = [], []

for iFile in range(len(file_names)):
    #print(file_path+file_names[iFile])
    infile = root.TFile.Open(file_path+file_names[iFile], "read")
    hists_read_file = []
    for ihist in range(N_hists):
        hists_read_file.append(infile.Get(hist_select_3D_names[ihist]))
        hists_read_file[-1].SetDirectory(root.nullptr)
        hists_read_file[-1].SetName(hists_read_file[-1].GetName()+f"{iFile}")
    hists_read.append(hists_read_file)


for iFile in range(len(sim_file_names)):
    #print(sim_file_path+sim_file_names[iFile])
    infile = root.TFile.Open(sim_file_path+sim_file_names[iFile], "read")
    hists_sim_file = []
    for ihist in range(len(hist_sim_3D_names)):
        hists_sim_file.append(infile.Get(hist_sim_3D_names[ihist]))
        hists_sim_file[-1].SetDirectory(root.nullptr)
        hists_sim_file[-1].SetName(hists_sim_file[-1].GetName()+f"_sim_{iFile}")
    hists_sim.append(hists_sim_file)

infile.Close()

In [6]:
type_loc = 3
pt = [1.95,4.45]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_sim[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_sim[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = projz.FindBin(central_bins[2*icentr+1])-1
    hist0 = hists_sim[0][type_loc].ProjectionX(f"hist0_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(hist0,title_x="(p_{T}^{RECO}-p_{T}^{TRUE})/p_{T}^{TRUE}",title_y="arbitrary norm",Lcolor=2,Lwidth=4,offset_y=1.1)
    hist1 = hists_sim[1][type_loc].ProjectionX(f"hist1_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(hist1,Lcolor=colors[0],Lwidth=4)
    hist0.Scale(1./(hist0.GetMaximum()+1e-32))
    hist1.Scale(1./(hist1.GetMaximum()+1e-32))
    root.gPad.SetLogy()
    hist0.GetXaxis().SetNdivisions(505)
    hist0.GetYaxis().SetRangeUser(2e-5,20)
    hist0.GetXaxis().SetRangeUser(-0.4,0.4)
    hist0.Draw("hist")
    hist1.Draw("same hist")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"{central_bins0[2*icentr]}-{central_bins0[2*icentr+1]}%"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(hist0,"brem corr","l")
    legends[-1].AddEntry(hist1,"uncorr","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v1.png")

In [7]:
iFile = 0
type_loc = 0
shift = 80
pt = [2.00,4.50]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(central_bins[2*icentr+0])
    ibinz1 = projz.FindBin(central_bins[2*icentr+1])-1
    real_proj = hists_read[iFile][type_loc].ProjectionX(f"real_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="#phi_{EMCAL}-#phi_{VTX}",title_y="arbitrary units",Lcolor=colors[0],Lwidth=4,offset_x=0.8)
    sim_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_proj_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_proj,Lcolor=root.kGreen+2,Lwidth=4)
    real_proj.Scale(1./(max(real_proj.GetMaximum(),max([real_proj.GetBinContent(45+i) for i in range(-3,12)]))))
    sim_proj.Scale(1./(sim_proj.GetMaximum()))
    real_proj.GetXaxis().SetRangeUser(-0.025,0.025)
    real_proj.GetYaxis().SetRangeUser(0,1.2)
    real_proj.Draw("hist")
    sim_proj.Draw("same hist")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"{central_bins0[2*icentr]}-{central_bins0[2*icentr+1]}%"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v0.png")

In [8]:
iFile = 0
type_loc = 1
shift = 80
pt = [2.00,4.50]
c0 = root.TCanvas(f"c0",f"c0",1450,900)
c0.Divide(3,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for icentr in range(N_centr):
    c0.cd(1+icentr)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    ibinz0 = projz.FindBin(central_bins[2*icentr+0]+shift)
    ibinz1 = projz.FindBin(central_bins[2*icentr+1]+shift)-1
    real_proj = hists_read[iFile][type_loc].ProjectionX(f"real_proj1_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(real_proj,title_x="#theta_{EMCAL}-#theta_{VTX}",title_y="arbitrary units",Lcolor=colors[0],Lwidth=4,offset_x=0.8)
    sim_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_proj1_{icentr}",ibiny0,ibiny1,ibinz0,ibinz1)
    Format_Hist_total(sim_proj,Lcolor=root.kGreen+2,Lwidth=4)
    real_proj.Scale(1./(max(real_proj.GetMaximum(),max([real_proj.GetBinContent(45+i) for i in range(-3,12)]))))
    sim_proj.Scale(1./(sim_proj.GetMaximum()))
    real_proj.GetYaxis().SetRangeUser(0,1.2)
    real_proj.GetXaxis().SetRangeUser(-0.025,0.025)
    real_proj.Draw("hist")
    sim_proj.Draw("same hist")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"{central_bins0[2*icentr]}-{central_bins0[2*icentr+1]}%"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v0.png")

In [9]:
iFile = 0
type_loc = 1
shift = 0
pt = [1.45,3.45]
c0 = root.TCanvas(f"c0",f"c0",1850,900)
c0.Divide(4,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
gauses = []
gauses_pol1 = []
for isec in range(8):
    c0.cd(1+isec)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    real_proj = hists_read[iFile][type_loc].ProjectionX(f"real_proj2_{isec}",ibiny0,ibiny1,1+2*isec + shift,1+2*isec+shift)
    sim_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_proj2_{isec}",ibiny0,ibiny1, 1+2*isec + shift,1+2*isec+shift)
    for icentr in range(1,N_centr):
        real_proj.Add(hists_read[iFile][type_loc].ProjectionX(f"real_proj2_{isec}_{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec + shift,16*icentr + 1+2*isec + shift) )
        sim_proj.Add(  hists_sim[iFile][type_loc].ProjectionX(f"sim_proj2_{isec}__{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec + shift,16*icentr + 1+2*isec + shift) )
        real_proj.Add(hists_read[iFile][type_loc].ProjectionX(f"real_proj2_{isec}_{icentr}",ibiny0,ibiny1,16*icentr + 2+2*isec + shift,16*icentr + 2+2*isec + shift) )
        sim_proj.Add(  hists_sim[iFile][type_loc].ProjectionX(f"sim_proj2_{isec}__{icentr}",ibiny0,ibiny1,16*icentr + 2+2*isec + shift,16*icentr + 2+2*isec + shift) )
        
    Format_Hist_total(real_proj,title_x="#theta_{EMCAL}-#theta_{VTX}",title_y="arbitrary units",Lcolor=colors[0],Lwidth=4,offset_x=0.8)
    Format_Hist_total(sim_proj,Lcolor=root.kGreen+2,Lwidth=4)
    real_proj.Scale(1./(real_proj.GetMaximum()))
    sim_proj.Scale(1./(sim_proj.GetMaximum()))
    real_proj.GetYaxis().SetRangeUser(0,1.2)
    real_proj.GetXaxis().SetRangeUser(-0.025,0.025)
    real_proj.Draw("hist")
    sim_proj.Draw("same hist")
    gauses.append(root.TF1(f"gaus_{isec}","gaus",-0.025,0.025)); gauses[-1].SetLineWidth(0)
    real_proj.Fit(gauses[-1],"RQ")
    gauses_pol1.append(root.TF1(f"gaus+pol1_{isec}","gaus(0)+pol1(3)",-0.025,0.025))
    gauses_pol1[-1].SetParameters(gauses[-1].GetParameter(0),gauses[-1].GetParameter(1),gauses[-1].GetParameter(2),0,0)
    real_proj.Fit(gauses_pol1[-1],"RQ+")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"Sector: {isec}"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v0.png")

In [10]:
print(f"const float theta_mean__emcal_sector_offsets[{N_sectors}] = {{{', '.join([str(round(gauses_pol1[i].GetParameter(1), 4)) for i in range(N_sectors)])}}};")
print(f"const float theta_sigma_emcal_sector_offsets[{N_sectors}] = {{{', '.join([str(round(gauses_pol1[i].GetParameter(2), 4)) for i in range(N_sectors)])}}};")

const float theta_mean__emcal_sector_offsets[8] = {-0.0001, -0.0003, -0.0002, -0.0004, 0.0001, 0.0001, -0.0001, 0.0004};
const float theta_sigma_emcal_sector_offsets[8] = {0.0035, 0.0035, 0.0037, 0.0036, 0.0035, 0.0035, 0.0035, 0.0033};


In [11]:
iFile = 0
type_loc = 0
shift = 80
pt = [1.45,1.85]
c0 = root.TCanvas(f"c0",f"c0",1850,900)
c0.Divide(4,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for isec in range(8):
    c0.cd(1+isec)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    real_proj = hists_read[iFile][type_loc].ProjectionX(f"real_proj4_{isec}",ibiny0,ibiny1,1+2*isec+shift,1+2*isec+shift)
    sim_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_proj4_{isec}",ibiny0,ibiny1, 1+2*isec+shift,1+2*isec+shift)
    for icentr in range(1,5):
        real_proj.Add(hists_read[iFile][type_loc].ProjectionX(f"real_proj4_{isec}_{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec+shift,16*icentr + 1+2*isec+shift) )
        sim_proj.Add(  hists_sim[iFile][type_loc].ProjectionX(f"sim_proj4_{isec}__{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec+shift,16*icentr + 1+2*isec+shift) )

    Format_Hist_total(real_proj,title_x="#phi_{EMCAL}-#phi_{VTX}",title_y="arbitrary units",Lcolor=colors[0],Lwidth=4,offset_x=0.8)
    Format_Hist_total(sim_proj,Lcolor=root.kGreen+2,Lwidth=4)
    real_proj.Scale(1./(real_proj.GetMaximum()))
    sim_proj.Scale(1./(sim_proj.GetMaximum()))
    real_proj.GetYaxis().SetRangeUser(0,1.2)
    real_proj.GetXaxis().SetRangeUser(-0.025,0.025)
    real_proj.Draw("hist")
    sim_proj.Draw("same hist")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"Sector: {isec}"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v0.png")

In [12]:
iFile = 0
type_loc = 0
shift = 80
icharge = 1
pt = [1.45,1.85]
c0 = root.TCanvas(f"c0",f"c0",1850,900)
c0.Divide(4,2)
projy = hists_read[0][type_loc].ProjectionY(f"projy{type_loc}")
projz = hists_read[0][type_loc].ProjectionZ(f"projz{type_loc}")
legends = []
for isec in range(8):
    c0.cd(1+isec)
    ibiny0 = projy.FindBin(pt[0])
    ibiny1 = projy.FindBin(pt[1])-1
    real_proj = hists_read[iFile][type_loc].ProjectionX(f"real_proj5_{isec}",ibiny0,ibiny1,1+2*isec+shift,1+2*isec+shift)
    sim_proj = hists_sim[iFile][type_loc].ProjectionX(f"sim_proj5_{isec}",ibiny0,ibiny1, 1+2*isec+shift,1+2*isec+shift)
    for icentr in range(1,5):
        real_proj.Add(hists_read[iFile][type_loc].ProjectionX(f"real_proj5_{isec}_{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec+ icharge + shift, 16*icentr + 1+2*isec+ icharge + shift) )
        sim_proj.Add(  hists_sim[iFile][type_loc].ProjectionX(f"sim_proj5_{isec}__{icentr}",ibiny0,ibiny1,16*icentr + 1+2*isec+ icharge + shift, 16*icentr + 1+2*isec+ icharge + shift) )
        
    Format_Hist_total(real_proj,title_x="#phi_{EMCAL}-#phi_{VTX}",title_y="arbitrary units",Lcolor=colors[0],Lwidth=4,offset_x=0.8)
    Format_Hist_total(sim_proj,Lcolor=root.kGreen+2,Lwidth=4)
    real_proj.Scale(1./(real_proj.GetMaximum()))
    sim_proj.Scale(1./(sim_proj.GetMaximum()))
    real_proj.GetYaxis().SetRangeUser(0,1.2)
    real_proj.GetXaxis().SetRangeUser(-0.025,0.025)
    real_proj.Draw("hist")
    sim_proj.Draw("same hist")
    legends.append(root.TLegend(0.25,0.85,0.85,0.95,f"Sector: {isec}"+", p_{T}"+f"={pt[0]}"))
    Format_Legend(legends[-1]); legends[-1].Draw()
    legends.append(root.TLegend(0.65,0.65,0.99,0.85,f""))
    #legends[-1].SetNColumns(2)
    legends[-1].AddEntry(real_proj,"real","l")
    legends[-1].AddEntry(sim_proj,"sim","l")
    Format_Legend(legends[-1])
    legends[-1].Draw()

c0.Draw()
c0.SaveAs(f"output/rich_emcal_qa/bremss_v0.png")